In [ ]:
# calculates NDVI from photo

In [2]:
%matplotlib inline
import pandas as pd
import xarray as xr
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO
import glob
import math
import scipy
import sklearn


In [39]:
# shows visual photo

vis_photo = 'https://phenocam.sr.unh.edu/data/archive/bozeman/2017/06/bozeman_2017_06_08_184805.jpg'
response = requests.get(vis_photo)
VIS_img = Image.open(BytesIO(response.content))
VIS_img_exp = 159

In [40]:
# finds all pixel values for visual image

width, height = VIS_img.size
vis_pixel_values = list(VIS_img.getdata())
vis_pixel_values = np.array(vis_pixel_values).reshape((width, height, 3))

In [26]:
# IR photo

ir_photo = 'https://phenocam.sr.unh.edu/data/archive/bozeman/2017/06/bozeman_IR_2017_06_08_201805.jpg'
response = requests.get(ir_photo)
IR_img = Image.open(BytesIO(response.content))
IR_img_exp = 2193

In [41]:
# finds all pixel values for IR image

width, height = IR_img.size
IR_pixel_values = list(IR_img.getdata())
IR_pixel_values = np.array(IR_pixel_values).reshape((width, height, 3))


In [50]:
# calculating all red values 

x=0
y=0
width,height = VIS_img.size
red_array = np.zeros((width,height))
for x in range (width):
    for y in range (height):
        red_array[x,y] = vis_pixel_values[x,y,0]

In [45]:
# finding r'

r_prime = red_array/math.sqrt(VIS_img_exp)

In [51]:
# calculating all green values 

x=0
y=0
width,height = VIS_img.size
green_array = np.zeros((width,height))
for x in range (width):
    for y in range (height):
        green_array[x,y] = vis_pixel_values[x,y,1]

In [55]:
# calculating all blue values

x=0
y=0
width,height = VIS_img.size
blue_array = np.zeros((width,height))
for x in range (width):
    for y in range (height):
        blue_array[x,y] = vis_pixel_values[x,y,2]

In [62]:
# calculating all IR values

x=0
y=0
width,height = IR_img.size
IR_array = np.zeros((width,height))
for x in range (width):
    for y in range (height):
        IR_array[x,y] = IR_pixel_values[x,y,0]

In [64]:
# finding z'

z = (red_array + green_array + blue_array) + IR_array

z_prime = z/math.sqrt(IR_img_exp)

In [71]:
# finding y'

y_array = 0.30*red_array + 0.59*green_array + 0.11*blue_array
y_prime = y_array/math.sqrt(IR_img_exp)

In [72]:
# finding x'   X′ = Z′ −Y′   z'=z/sqrt(exposure of RGB+IR)

x_prime = z_prime - y_prime

In [73]:
# camera NDVI

camera_ndvi = ((x_prime - r_prime)/(x_prime + r_prime))

/Users/BillVandenberg/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()
